### **이미지가 포함된 보고서 생성그래프 만들기**

In [1]:
!pip install -U --q langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 134.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 150.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s e

In [2]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.0 MB/s eta 0:00:00


#### **API 키 설정** ####

In [3]:
import os
import warnings
warnings.filterwarnings('ignore')

from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('tavily')

#### **Graph State 설정** ####

In [4]:
# =====================
# 기본 import / 타입 정의
# =====================
from typing import Annotated, TypedDict, List, Dict, Sequence, Optional

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from pydantic import Field, create_model
from openai import OpenAI

from docx import Document
from docx.shared import Inches
import requests
from io import BytesIO

from IPython.display import Image, display

# =====================
# LLM / Tool 초기화
# =====================
client = OpenAI()
llm = ChatOpenAI(model="gpt-4o-mini")
search = TavilySearchResults(max_results=3)

# =====================
# State 정의
# =====================
class State(TypedDict):
    # LangGraph messages
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # "section1", "section2", ... : 제목
    outline: Dict[str, str]
    # 현재 섹션 번호
    current_section: int
    # 현재 섹션 본문 내용
    section_content: str
    # 현재 섹션 이미지 URL
    section_image: str
    # 이미지 프롬프트
    image_prompt: str
    # 전체 섹션 수
    total_sections: int
    # 전체 보고서: [{title, content, image_url, image_prompt}, ...]
    full_report: List[Dict[str, str]]
    # (선택) 에러 메시지
    # error: Optional[str]

# =====================
# 1. 아웃라인 동적 모델 생성
# =====================
def create_outline_model(section_count: int):
    fields = {
        f"section{i}": (str, Field(description=f"Title for section {i}"))
        for i in range(1, section_count + 1)
    }
    return create_model("DynamicOutline", **fields)

# =====================
# 2. 아웃라인 생성 노드
# =====================
def outline_generator(state: State):
    DynamicOutline = create_outline_model(state["total_sections"])
    outline_parser = JsonOutputParser(pydantic_object=DynamicOutline)

    outline_prompt = PromptTemplate(
        template="""
        아래 주제에 대해 {section_count}개의 주요 섹션으로 구성된 보고서 개요를 작성해줘.
        각 섹션은 간결하고 명확한 제목으로 구성해줘.

        출력 형식 안내:
        {format_instructions}

        주제: {topic}
        """.strip(),
        input_variables=["section_count", "topic"],
        partial_variables={
            "format_instructions": outline_parser.get_format_instructions()
        }
    )

    chain = outline_prompt | llm | outline_parser

    outline_obj = chain.invoke({
        "section_count": state["total_sections"],
        "topic": state["messages"][-1].content
    })

    # JsonOutputParser + pydantic_object → pydantic 모델 or dict
    if hasattr(outline_obj, "model_dump"):
        outline = outline_obj.model_dump()
    elif hasattr(outline_obj, "dict"):
        outline = outline_obj.dict()
    else:
        outline = dict(outline_obj)

    return {"outline": outline}

# =====================
# 3. 본문 작성 노드
# =====================
def contents_writer(state: State):
    # 에러 핸들링
    if "error" in state:
        return {"messages": [AIMessage(content=f"An error occurred: {state['error']}")]}

    # 모든 섹션 완료 시
    if state["current_section"] > state["total_sections"]:
        return {"messages": [AIMessage(content="Report completed.")]}

    current_section_key = f"section{state['current_section']}"
    current_topic = state["outline"][current_section_key]

    # Tavily 검색
    search_results = search.invoke(current_topic)

    # 이전 섹션들 내용(full_report 기준)
    previous_sections_content = []
    for i, section in enumerate(state.get("full_report", []), start=1):
        previous_sections_content.append(
            f"Section {i}:\n{section['title']}\n{section['content']}"
        )
    previous_sections = "\n\n".join(previous_sections_content) if previous_sections_content else "없음"

    section_prompt = PromptTemplate(
        template="""
        주제: {topic} 에 대한 자세한 보고서 섹션을 작성해줘.

        아래는 해당 주제와 관련하여 수집된 검색 결과야:
        {search_results}

        이전에 작성된 섹션 내용은 다음과 같아:
        {previous_sections}

        이 섹션에는 이미지 생성 프롬프트나 제안은 포함하지마.
        수치 정보나 구체적인 설명이 필요하므로,
        위의 검색 결과에서 얻은 자료를 최대한 활용해줘.
        """.strip(),
        input_variables=["topic", "search_results", "previous_sections"],
    )

    section_msg = llm.invoke(section_prompt.format(
        topic=current_topic,
        search_results=search_results,
        previous_sections=previous_sections
    ))

    return {
        "section_content": section_msg.content,
        # current_section는 그대로 유지 (image_generator에서 +1 해 줌)
        "current_section": state["current_section"]
    }

# =====================
# 4. 이미지 생성 유틸 함수
# =====================
def generate_image(prompt: str) -> str:
    """DALL-E를 이용해 프롬프트 기반 이미지를 생성합니다."""
    try:
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1
        )
        return response.data[0].url
    except Exception as e:
        print("이미지 생성 실패:", e)
        return "Image generation failed"

# =====================
# 5. 이미지 생성 노드
# =====================
def image_generator(state: State):
    prompt_template = PromptTemplate(
        template="""
        아래의 섹션 내용을 시각적으로 표현할 수 있는 인포그래픽을 만들기 위한 프롬프트를 작성해줘.
        프롬프트는 500자 이내로 간결하고 명확하게 구성해줘.

        섹션 내용:

        {section_content}

        생성할 이미지에 대한 프롬프트:
        """.strip(),
        input_variables=["section_content"],
    )

    image_prompt_msg = llm.invoke(
        prompt_template.format(section_content=state["section_content"])
    )

    image_prompt_text = (
        image_prompt_msg.content
        if isinstance(image_prompt_msg, AIMessage)
        else str(image_prompt_msg)
    )

    image_url = generate_image(image_prompt_text)

    current_section_index = state["current_section"]
    title = state["outline"][f"section{current_section_index}"]

    current_section = {
        "title": title,
        "content": state["section_content"],
        "image_url": image_url,
        "image_prompt": image_prompt_text,
    }

    updated_full_report = state.get("full_report", []) + [current_section]

    print(f"{state['total_sections']}개 중 {current_section_index}번 섹션 생성 완료")

    return {
        "image_prompt": image_prompt_text,
        "section_image": image_url,
        "current_section": current_section_index + 1,  # 다음 섹션으로
        "full_report": updated_full_report,
    }

/tmp/ipython-input-3651376850.py:30: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=3)


#### **웹 검색 도구 정의** ####

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=3)

#### **개요 작성 에이전트 설정** ####

In [6]:
from pydantic import BaseModel, Field, create_model

# 사용자가 원하는 섹션 개수에 따라 동적으로 Pydantic 모델을 생성하는 함수
def create_outline_model(section_count: int):
    # 딕셔너리 컴프리헨션을 통해 section1, section2, ..., section{n} 형태의 필드를 생성
    # 각 필드는 문자열(str) 타입이며, Field를 사용하여 설명(description)을 부여함
    fields = {
        f"section{i}": (str, Field(description=f"Title for section {i}"))
        for i in range(1, section_count + 1)
    }

    # create_model은 런타임에 새로운 Pydantic 모델 클래스를 생성하는 함수
    # 첫 번째 인자는 모델의 이름, 이후는 필드들을 키워드 인자(**fields)로 전달
    return create_model("DynamicOutline", **fields)

In [7]:
def outline_generator(state: State):
    # 섹션 수에 맞는 동적 Pydantic 모델 생성
    DynamicOutline = create_outline_model(state["total_sections"])
    outline_parser = JsonOutputParser(pydantic_object=DynamicOutline)

    outline_prompt = PromptTemplate(
        template="""
        아래 주제에 대해 {section_count}개의 주요 섹션으로 구성된 보고서 개요를 작성해줘.
        각 섹션은 간결하고 명확한 제목으로 구성해줘.

        출력 형식 안내:
        {format_instructions}

        주제: {topic}
        """,
        input_variables=["section_count", "topic"],
        partial_variables={
            "format_instructions": outline_parser.get_format_instructions()
        }
    )

    # 프롬프트 → LLM → 파서로 연결된 체인 구성
    chain = outline_prompt | llm | outline_parser

    # 실행: 사용자 메시지를 토픽으로 사용
    outline = chain.invoke({
        "section_count": state["total_sections"],
        "topic": state["messages"][-1].content
    })

    return {"outline": outline}

#### **이미지 생성 에이전트 설정** ####

In [8]:
from openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import AIMessage  # 필요 시 사용
from langchain_core.tools import tool

client = OpenAI()

# 이미지 생성 함수
def generate_image(prompt):
    """DALL-E를 이용해 프롬프트 기반 이미지를 생성합니다."""
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",  # 단위: pixel
        quality="standard",
        n=1
    )
    return response.data[0].url

# 이미지 생성 노드
def image_generator(state: State):
    prompt_template = PromptTemplate(
        template="""
        아래의 섹션 내용을 시각적으로 표현할 수 있는 인포그래픽을 만들기 위한 프롬프트를 작성해줘.
        프롬프트는 500자 이내로 간결하고 명확하게 구성해줘.

        섹션 내용:

        {section_content}

        생성할 이미지에 대한 프롬프트:""",
        input_variables=["section_content"],
    )

    # 프롬프트 포맷팅 후 LLM에 전달
    image_prompt = llm.invoke(prompt_template.format(section_content=state["section_content"]))

    # 이미지 생성
    image_url = generate_image(image_prompt.content)

    # 현재 섹션 정보를 저장
    current_section = {
        "title": state['outline'][f"section{state['current_section']}"],
        "content": state['section_content'],
        "image_url": image_url,
        "image_prompt": image_prompt.content if isinstance(image_prompt, AIMessage) else image_prompt
    }

    # 전체 보고서에 추가
    updated_full_report = state.get("full_report", []) + [current_section]

    print(f"{state['total_sections']}개 중 {state['current_section']}번 섹션 생성 완료")

    # 다음 섹션으로 넘어가며 상태 갱신
    return {
        "image_prompt": image_prompt.content if isinstance(image_prompt, AIMessage) else image_prompt,
        "section_image": image_url,
        "current_section": state["current_section"] + 1,
        "full_report": updated_full_report
    }


ModuleNotFoundError: No module named 'langchain.prompts'

#### **LLM 설정** ####

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

#### **콘텐츠 작성 에이전트 설정** ####

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

def contents_writer(state: State):
    if "error" in state:
        return {"messages": [AIMessage(content=f"An error occurred: {state['error']}")]}

    # 현재 섹션이 전체 섹션보다 크면 >> 보고서 완성
    if state["current_section"] > state["total_sections"]:
        return {"messages": [AIMessage(content="Report completed.")]}

    current_section_key = f"section{state['current_section']}"
    current_topic = state["outline"][current_section_key]
    search_results = search.invoke(current_topic)

    previous_sections_content = []
    for i in range(1, state['current_section']):
        section_key = f"section{i}"
        if section_key in state["section_content"]:
            previous_sections_content.append(f"""
            Section {i}:
            {state['outline'][section_key]}
            {state['section_content'][section_key]}
            """)

    previous_sections = "\n\n".join(previous_sections_content)

    section_prompt = PromptTemplate(
    template="""
    주제: {topic} 에 대한 자세한 보고서 섹션을 작성해줘.

    아래는 해당 주제와 관련하여 수집된 검색 결과야:
    {search_results}

    이전에 작성된 섹션 내용은 다음과 같아:
    {previous_sections}

    이 섹션에는 이미지 생성 프롬프트나 제안은 포함하지마.
    수치 정보나 구체적인 설명이 필요하므로,
    위의 검색 결과에서 얻은 자료를 최대한 활용해줘.
    """,
    input_variables=["topic", "search_results", "previous_sections"],
    )

    section_content = llm.invoke(section_prompt.format(
        topic=current_topic,
        search_results=search_results,
        previous_sections=previous_sections
    ))

    return {
        "section_content": section_content.content,
        "current_section": state["current_section"]
    }


#### **워드 생성 에이전트 설정** ####

In [ ]:
from docx import Document
from docx.shared import Inches
import requests
from io import BytesIO #바이트 인풋 아웃풋

def report_generator(state: State):
    doc = Document()
    doc.add_heading(f"Report: {state['messages'][0].content}", 0)
    # 제목 사용자가 처음 입력한 메시지(['messages'[0]])

    for section in state['full_report']:
        doc.add_heading(section['title'], level=1)
        doc.add_paragraph(section['content'])

        # 이미지 추가
        if section['image_url'] != "Image generation failed":
            try:
                response = requests.get(section['image_url']) # 이미지 가져오기(image_url)
                image = BytesIO(response.content)             # 가져온 이미지 decoding
                doc.add_picture(image, width=Inches(6))       # 이미지 추가
                doc.add_paragraph(f"Image prompt: {section['image_prompt']}")
            except Exception as e:
                doc.add_paragraph(f"Failed to add image: {str(e)}")

        doc.add_page_break()
        # 다음 페이지로 넘어가게 정지조건 설정

    # 보고서 저장
    filename = f"report_{state['messages'][0].content}.docx".replace(" ", "_")
    doc.save(filename)

    return {
        "messages": [AIMessage(content=f"Report finalized and saved as {filename}.")],
        "report_file": filename
    }


#### **그래프 구축** ####

In [ ]:
# 노드 추가
graph_builder.add_node("outline_generator", outline_generator)  # 아웃라인 생성 노드
graph_builder.add_node("contents_writer", contents_writer)      # 본문 작성 노드
graph_builder.add_node("image_generator", image_generator)      # 이미지 생성 노드
graph_builder.add_node("report_generator", report_generator)    # 최종 보고서 생성 노드

# 엣지(흐름 연결) 추가
graph_builder.add_edge(START, "outline_generator")              # 시작 → 아웃라인 생성
graph_builder.add_edge("outline_generator", "contents_writer")  # 아웃라인 생성 → 본문 작성
graph_builder.add_edge("contents_writer", "image_generator")    # 본문 작성 → 이미지 생성
graph_builder.add_edge("report_generator", END)                 # 보고서 생성 → 종료

# 조건부 엣지(분기 조건 정의)
def should_continue_writing(state: State):
    # 현재 섹션이 전체 섹션 수 이하라면 계속 작성
    if state["current_section"] <= state["total_sections"]:
        return "write_section"
    # 그렇지 않다면 보고서 마무리 단계로 이동
    else:
        return "finalize_report"

graph_builder.add_conditional_edges(
    "image_generator",          # 분기 시작 지점: 이미지 생성 이후
    should_continue_writing,    # 분기 조건 함수
    {
        "write_section": "contents_writer",   # 계속 작성 → 본문 작성 노드로 돌아감
        "finalize_report": "report_generator" # 마무리 단계 → 보고서 생성으로 이동
    }
)

# 그래프 컴파일
graph = graph_builder.compile()

#### **그래프 시각화**

In [ ]:
from IPython.display import Image, display

# PNG 시도
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"PNG 출력 실패: {e}")
    # Mermaid 텍스트 직접 출력
    print(graph.get_graph().draw_mermaid())


#### **랭그래프 실행**

In [ ]:
from langchain_core.messages import HumanMessage

# 사용자 입력 받기
topic = input("보고서 주제를 입력하세요: ")
total_sections = int(input("생성할 섹션의 수를 입력하세요: "))

# 초기 상태 설정
initial_state = {
    "messages": [HumanMessage(content=topic)],
    "total_sections": total_sections,
    "current_section": 1,
}

# 그래프 실행
for chunk in graph.stream(initial_state,stream_mode="update"):
    print(chunk)

print("\n=== 보고서 생성 완료 ===")